# Demo notebook: anomaly detection

This is a demo code to show the functionality of the balance anomaly detection engine in the Relevant Facts project.

# 1. Dependencies and configuration

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pnd
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15,7]

import seaborn as sns

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
import sys
import os

sys.path.insert(0, '/opt/cloudera/parcels/SPARK2/lib/spark2/python')
#sys.path.insert(0, '.')
sys.path.insert(0, '/opt/cloudera/parcels/SPARK2/lib/spark2/python/lib/py4j-0.10.7-src.zip')
sys.path.insert(0, '/us/e055324/hechos_relevantes/rf_16_demo/da_srm_relevant_facts')


os.environ['PYSPARK_PYTHON'] = '/DYA/da_tech/envs/advice_pfm/bin/python'
os.environ["PYSPARK_DRIVER_PYTHON"] = '/DYA/da_tech/envs/advice_pfm/bin/python'

In [ ]:
import getpass
pw = getpass.getpass()
!kinit <<< $pw 

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext
from pyspark.sql import SQLContext

CONF = (pyspark.SparkConf()
            .setMaster("yarn")
            .setAppName("AIConf_demo_HR"))

CONF = CONF.set("spark.executor.memory", "1g") \
    .set("spark.driver.memory", "1g") \
    .set("spark.dynamicAllocation.enabled", "True") \
    .set("spark.executor.cores", "2") \
    .set("spark.yarn.queue", "root.DyA.DS") \
    .set("spark.executor.memoryOverhead", "1024") \
    .set("spark.driver.memoryOverhead", "1024")\
    .set("spark.port.maxRetries", 100) \
    .set('spark.dynamicAllocation.maxExecutors', 5) \
    .set('spark.dynamicAllocation.minExecutors', 3) \
    .set('spark.sql.sources.partitionOverwriteMode', "dynamic")

spark = SparkSession.builder.config(conf=CONF).getOrCreate()
hiveContext = HiveContext(spark)
sqlContext = SQLContext(spark)

### Internal dependencies and configuration

The code in this demo uses the pre-productive version of Relevant Fact 16 (RF16) and the productive model for balance forecasting:

In [ ]:
from pyspark.sql.types import TimestampType
from pyhocon import ConfigFactory
from datetime import datetime
from typing import List
from dateutil.relativedelta import relativedelta
import hr.datasources.transactions as transactions
from hr.datasources.operations.transactions_sql_operations import transactions_between_interval
from hr.datasources.operations.transactions_sql_operations import filter_titularity, filter_visible_product
from hr.datasources.operations.transactions_sql_operations import filter_commercial_communications
from pyspark.sql.functions import col, abs, size, udf, lit
from pyspark.sql.types import ArrayType, DoubleType, IntegerType
import numpy as np
import pandas as pd
from pyspark.sql import Window
from pyspark.sql.functions import row_number
from hr.daily.rf16_balance_anomaly.rf16_balance_anomaly import RF16
import hr.datasources.clients as clients
from hr.datasources.operations.transactions_sql_operations import join_clients_with_transactions
from hr.datasources.ts_balanced import TsBalanced
#from hr.forecasting_balance.forecasting_transform import ForecastingBalanceTransform

spark.sparkContext.addPyFile("/us/e055324/hechos_relevantes/rf_16_demo/da_srm_relevant_facts_dist/hr_dist.zip")
#spark.sparkContext.addPyFile("/us/e043894/relevant_facts/release_0_0_10/hr_dist.zip")
#path_model = "/us/e055324/hechos_relevantes/develop/da_srm_relevant_facts/hr/model/11q_model"

CUSTOM_DATA_DEPENDENCIES = "CUSTOM_DATA_DEPENDENCIES"
custom_data_config = {}
custom_data_config.update({CUSTOM_DATA_DEPENDENCIES:
                           ConfigFactory.parse_file("/us/e055324/hechos_relevantes/rf_16_demo/da_srm_relevant_facts/hr/datasources/conf/datasources_dependencies_dev.json")})

day = datetime.strptime("20191003", "%Y%m%d")
yesterday = day - relativedelta(days=1)
start_day=day - relativedelta(months=6)
end_day=day - relativedelta(days=1)
norm_tolerance = 0.01
k_low = 4
k_high = 4
thr_variation_ratio_main_mov = 0.9
lag = 0
win_l = 1
        
categorizer_output_path = (custom_data_config[CUSTOM_DATA_DEPENDENCIES]["transactions"]["movements_categoriser_output"]["hdfs_path"].format(env="dev", release_type="release_candidate"))
execution_plan={'RF16': [datetime(2019, 10, 3, 0, 0)]}

dataset_transactions_opath = "da_hechos_dev.tc_dataset_demo"
dataset_ts_balance_opath = "da_hechos_dev.ts_dataset_demo"
dataset_balance_pred_opath = "da_hechos_dev.bp_dataset_demo"

### Demo dataset

We have defined a reduced dataset that comprises some illustrative series. This chunk filters the corresponding transactions, composes time series and obtains balance forecastings:

In [ ]:
transactions_clients_df = spark.read.table(dataset_transactions_opath).persist()
ts_balanced = spark.read.table(dataset_ts_balance_opath).persist()
balance_with_prediction = spark.read.table(dataset_balance_pred_opath).persist()

# 2. Functions to analyze and visualize results

In [ ]:
from ipywidgets import interact, widgets

In [ ]:
import pyspark.sql.functions as sf

def plot_anomalous_series (cod_idcontra, df_results):
    
    n_res = df_results.filter(sf.col('cod_idcontra')==cod_idcontra).count()
    
    fig, (ax1) = plt.subplots(1, 1)

    plt.rcParams['figure.figsize'] = [15,5]
    
    if (n_res == 0):
        print('No series!')
        return
    
    pnd_sample_ts = (df_results
                     .filter(sf.col('cod_idcontra')==cod_idcontra)
                     .select(['time_series','predict','thr_low', 'thr_high'])
                     .toPandas()
                    )
    
    

    ax1.plot(range(len(pnd_sample_ts.time_series[0])),
                     pnd_sample_ts.time_series[0],'-o')


    ax1.plot(range(len(pnd_sample_ts.time_series[0][0:-1]+pnd_sample_ts.predict[0][5])),
                     pnd_sample_ts.time_series[0][0:-1]+pnd_sample_ts.predict[0][5],'--k')

    ax1.legend(['Real (current)','Real (prev) + Median'])
    ax1.set_ylabel('Balance (€)')
    ax1.set_xlabel('Day')

    ax1.axhspan(pnd_sample_ts.thr_low[0][0], pnd_sample_ts.thr_high[0][0],color='green',alpha=.1)
    
    for i in range(5):
        ax1.fill_between(range(len(pnd_sample_ts.time_series[0][0:-1]+pnd_sample_ts.predict[0][i])),
                         pnd_sample_ts.time_series[0][0:-1]+pnd_sample_ts.predict[0][i],
                         pnd_sample_ts.time_series[0][0:-1]+pnd_sample_ts.predict[0][10-i],
                        alpha=.1,
                        color='blue')
        
    
    return ax1

# 3. Anomaly detection in series

We use a threshold-based anomaly detection approach that relies on the quantile regression results. 
Specifically, our engine raises an alarm if

\begin{align}
(\mathbb{P}\{Balance(t) < X_t\} < Thr_{min}) \lor (\mathbb{P}\{Balance(t) > X_t\} < Thr_{max})
\end{align}

where $Balance(t)$ is the predicted quantiles for time $t$, $X_t$ is the series value at time $t$, and $(Thr_{min}+ Thr_{max})$ is the anomalous range's probability. To offer noticeable results without overwhelming clients, we heuristically control $(Thr_{min}+ Thr_{max})$ by considering as anomalies those observations outside an interval of the form:

\begin{align}
[Q_5 − k_1\frac{(Q_{50}-Q_5)}{(Q_{95}-Q_5)}(Q_{95}-Q_5) , Q_{95} + k_2(1-\frac{(Q_{50}-Q_5)}{(Q_{95}-Q_5)})(Q_{95}-Q_5)]
\end{align}

which is inspired in the proposal by Hubert & Vandervieren (2008) to adapt the well-known Tukey criterion to asymmetric distributions.

Here, both $k_1$ and $k_2$ allow introducing differential weights for abnormal values above / below the non-atypical values' range. Note that, as Hinkley's coefficient 

\begin{align}
0 = \frac{((Q_{95}-Q_{50}) - (Q_{50}-Q_{5}))}{(Q_{95}-Q_{5})}
\end{align}

is 0 for symmetric distributions,
if we scale $ [Q_5, Q_{95}] $ to $[0,1]$, then 

\begin{align}
|Q_{50} - Q_5|+|Q_{95} - Q_{50}| = 1
\end{align}

and 

\begin{align}
|Q_{50} - Q_5| \rightarrow 0.5, |Q_{95} - Q_{50}| \rightarrow 0.5
\end{align}

when the distribution tends to be symmetric.

With $k_1 = k_2 = 4$, we have that for fairly symmetric distributions:

\begin{align}
[Q_5 − 2(Q_{95}-Q_5) , Q_{95} + 2(Q_{95}-Q_5)]
\end{align}

which empirically showed good results in terms of detected events (it may be somehow restrictive in case of Gaussian distributions, but suits the heavier-tailed nature of balance series).
Anyhow, these parameters can be tuned to reflect a less restrictive behavior, or to introduce some bias towards the detection of negative or positive divergences.

<cite>Hubert, M., & Vandervieren, E. (2008). An adjusted boxplot for skewed distributions. Computational statistics & data analysis, 52(12), 5186-5201.</cite>

In [ ]:
@interact
def plot_graf(k1=[4,0,1,10],
              k2=[4,0,1,10],
             cod_idcontra = ['00000000000000000000XXXXXX',
                             '00000000000000000000XXXXXX',
                             '00000000000000000000XXXXXX']):
    
    
    rf16_transform = RF16(day=day)
    
    rf16_transform.k_high = k2
    rf16_transform.k_low = k1
    
    df_anomalies = rf16_transform.obtain_rfs_tosend(transactions_clients_df, 
                                                ts_balanced, 
                                                balance_with_prediction.drop('time_series'))
    
    plot_anomalous_series (cod_idcontra, df_anomalies)

In [ ]:
spark.stop()